In [ ]:
"""
imdb데이터의 EDA입니다
"""

In [14]:
import pandas as pd

data = pd.read_csv('review_db.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
#one hot encoding
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [16]:
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [17]:
import re

data['review'] = data['review'].str.replace('<br />',' ')
data['review'] = data['review'].apply(lambda x:clean_text(x))
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming te...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [18]:
from collections import Counter

data['temp_review'] = data['review'].apply(lambda x:str(x).split())
top = Counter([item for sublist in data['temp_review'] for item in sublist])
temp = pd.DataFrame(top.most_common(15))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,the,663856
1,and,320701
2,a,320554
3,of,288463
4,to,266912
5,is,210506
6,in,185047
7,it,154901
8,i,152098
9,this,149895


In [19]:
import plotly.express as px

fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [20]:
import nltk
from nltk.corpus import stopwords

def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
data['temp_review'] = data['temp_review'].apply(lambda x:remove_stopword(x))

In [21]:
top = Counter([item for sublist in data['temp_review'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

,Common_words,count
1,film,76026
2,one,51472
3,like,39061
4,good,28912
5,even,24580
6,would,24034
7,time,23958
8,really,23011
9,see,22639
10,story,22512


In [22]:
Positive_sent = data[data['sentiment']==1]
Negative_sent = data[data['sentiment']==0]

#MosT common positive words
top = Counter([item for sublist in Positive_sent['temp_review'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

,Common_words,count
0,film,40161
1,movie,36700
2,one,26291
3,like,17180
4,good,14572
5,great,12810
6,story,12610
7,time,12122
8,see,11985
9,well,11406


In [23]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['temp_review'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

,Common_words,count
1,film,35865
2,one,25181
3,like,21881
4,even,15098
5,bad,14362
6,good,14340
7,would,13651
8,really,12293
9,time,11836
10,see,10654


In [33]:
raw_text = [word for word_list in data['temp_review'] for word in word_list]

In [36]:
def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        segment - Segment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_data[train_data.segments == segments]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in data[data.sentiment != sentiment]['temp_review']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in data[data.sentiment == sentiment]['temp_review']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

In [ ]:
Unique_Positive= words_unique(1, 20, raw_text)
print("The top 20 unique words in Positive reviews are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
Unique_Negative= words_unique(0, 10, raw_text)
print("The top 10 unique words in Negative reviews are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
"""

분석 데이터의 eda입니다

1. tweet 데이터

2. Quora 데이터

"""

In [27]:
#1. 트윗 데이터

tw_data0501 = pd.read_csv('twitter_data_2020-05-01_to_2020-05-09.csv', encoding = 'UTF-8')
tw_data0510 = pd.read_csv('twitter_data_2020-05-10_to_2020-05-20.csv', encoding = 'UTF-8')
tw_data0521 = pd.read_csv('twitter_data_2020-05-21_to_2020-05-30.csv', encoding = 'UTF-8')
tw_data0601 = pd.read_csv('twitter_data_2020-06-01_to_2020-06-10.csv', encoding = 'UTF-8')

tw_data = pd.concat([tw_data0501, tw_data0510, tw_data0521, tw_data0601])
tw_data.drop(['date', 'user_name'], axis='columns', inplace=True)

tw_data['text'] = tw_data['text'].str.replace('<br />',' ')
tw_data['text'] = tw_data['text'].apply(lambda x:clean_text(x))

tw_data.head()

,text
0,current trends in iot at workplace market is ...
1,iot plays an important role in pharmaceutical ...
2,microsoft shells out for iot security threat...
3,with billions billions of iot connected devic...
4,interesting discussion no doubt esim is the fu...


In [29]:
tw_data['temp_text'] = tw_data['text'].apply(lambda x:str(x).split())
tw_data['temp_text'] = tw_data['temp_text'].apply(lambda x:remove_stopword(x))

top = Counter([item for sublist in tw_data['temp_text'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,iot,14032
1,security,12453
2,cybersecurity,3063
3,devices,1588
4,iotsecurity,1511
5,data,1343
6,internetofthings,1290
7,privacy,1217
8,via,1169
9,ai,1156


In [31]:
#2. Quora 데이터

q_data = pd.read_csv('quora_data.csv', encoding = 'latin')
q_data.rename(columns = {'comments' : 'text'}, inplace = True)

q_data['text'] = q_data['text'].str.replace('<br />',' ')
q_data['text'] = q_data['text'].apply(lambda x:clean_text(x))

q_data.head()

,text
0,research study forecasts revenue from the glob...
1,download sample pdf to get more information ab...
2,government initiatives in smart cities projects
3,here is a list of some root cause for vulnerab...
4,according to new idc spending guide the worldw...


In [32]:
q_data['temp_text'] = q_data['text'].apply(lambda x:str(x).split())
q_data['temp_text'] = q_data['temp_text'].apply(lambda x:remove_stopword(x))

top = Counter([item for sublist in q_data['temp_text'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,iot,254
1,security,193
2,devices,74
3,internet,51
4,network,38
5,things,36
6,market,33
7,new,27
8,device,27
9,business,27
